# Set up database dimension for select link assignment

##Initial Script: Ziying Ouyang Feb 2019

In [8]:
import os

modeller = inro.modeller.Modeller()
emmebank = modeller.emmebank
change_db_dimension = modeller.tool("inro.emme.data.database.change_database_dimensions")
import_attr_values = modeller.tool("inro.emme.data.network.import_attribute_values")
create_attr = modeller.tool("inro.emme.data.extra_attribute.create_extra_attribute")
copy_attr = modeller.tool("inro.emme.data.network.copy_attribute")



In [2]:
# the CSV file is in emme_project/database
# with two columns: zone_id, group_id (tier 1 employment centers) 
scenario = emmebank.scenario(101)
create_attr("NODE", "@selected_zone", "jurisdiction", overwrite=True, scenario=scenario)

default_path = os.path.dirname(modeller.emmebank.path).replace("\\","/")
file_path = os.path.join(default_path,"xref_taz13_jur.csv").replace("\\","/")
import_attr_values(file_path,
              field_separator = ",",
              scenario = scenario,
              column_labels={0: "inode", 
                             1: "@selected_zone"},
              revert_on_error=True)

for scenario_id in range(102, 106):
    time_scenario = emmebank.scenario(scenario_id)
    create_attr("NODE", "@selected_zone", "jurisdiction", overwrite=True, scenario=time_scenario)
    copy_attr(from_attribute_name = "@selected_zone",
              to_attribute_name = "@selected_zone", 
              from_scenario = scenario, 
              to_scenario = time_scenario)


In [10]:
### based on number of selected regions in query set this number. 40000000 is the default for 4 regions
required_dimensions = 40000000
new_dimensions = emmebank.dimensions
if new_dimensions["extra_attribute_values"] < required_dimensions :
    new_dimensions["extra_attribute_values"] = required_dimensions
    change_db_dimension(emmebank_dimensions=new_dimensions,
              keep_backup=True)